In [1]:
%load_ext autoreload
%autoreload 2

In [452]:
from wdtools import *

In [32]:
#setID = 'Set001'; nm_to_add=0
#setID = 'Set002'; nm_to_add=1420
#setID = 'Set003'; nm_to_add = 2143
setID = 'Set004'; nm_to_add = 2878

In [33]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [59]:
unmapped_notes = pd.read_excel(f'{inpath}\\notes\\LCOGUnmappedRecordsJB.xlsx', skiprows=1)

In [13]:
mappedIDs = ['WD2017-0198', 'WD2017-0229', 'WD2019-0082', 'WD2021-0179']
holdIDs = ['WD2022-0291', 'WD2021-0703']
nomapIDs = ['WD2021-0181', 'WD2022-0158']
maybeIDs = ['WD2017-0050', 'WD2017-0022', 'WD2020-0531']

In [15]:
skipIDs = mappedIDs + holdIDs + nomapIDs + maybeIDs

In [16]:
wdIDs = [wid for wid in unmapped_notes.wetdet_delin_number.unique() if wid not in skipIDs]

In [34]:
len(wdIDs)

31

In [35]:
toCheck_df = wd[wd.wetdet_delin_number.isin(wdIDs)]

In [36]:
review_loop_r1(df=toCheck_df, partial=True, idx=True)

0.0% digitized, 15 records remained, expected to be done in about 12 hours...
WD2021-0031
index = 27
   county     trsqq parcel_id   latitude   longitude  record_ID notes  \
26  Crook  15S14E01   702,800  44.292086 -120.989141       2905         
27  Crook  15S15E07       300  44.292086 -120.989141       2906         
28  Crook  15S14E12   100,401  44.292086 -120.989141       2907         
29  Crook  15S14E06       600  44.292086 -120.989141       2908         

   missinglot  
26          N  
27          N  
28          N  
29          N  
Press 'p' to pause or any key to stop...p
Press 'a' to add the wd record or 'c' to continue...c
6.7% digitized, 14 records remained, expected to be done in about 11 hours...
WD2018-0002
index = 97
        county     trsqq parcel_id   latitude   longitude  record_ID  \
96  Hood River  03S09E00  100p,ROW  45.319481 -121.628633       2975   

           notes missinglot  
96  ROW, Partial          N  
Press 'p' to pause or any key to stop...p
Press 'a'

In [37]:
issuepath = inpath + '\\GIS\\ArcGIS Pro Project\\DataReview\\issueIDs.gdb'

In [369]:
mapped = [lyr for lyr in fiona.listlayers(issuepath) if (lyr != 'WD2017_0229') and ('L' not in lyr)]

In [40]:
len(mapped)

35

In [370]:
gdf = merge_single_partial_file(setID=False, wIDlist=mapped, from_set=False)

In [372]:
gdf = gdf.dissolve('wdID')
gdf['wdID'] = gdf.index
gdf.reset_index(drop=True, inplace=True)

In [373]:
code0 = ["WD2019-0300", "WD2021-0597", "WD2019-0436", "WD2020-0568", "WD2019-0300"]
code1 = ["WD2017-0314", "WD2019-0046", "WD2019-0496", "WD2021-0416", "WD2021-0653"]

In [374]:
gdf.loc[gdf.wdID.isin(code0),'code'] = 0
gdf.loc[gdf.wdID.isin(code1),'code'] = 1
gdf.loc[gdf.code.isnull(),'code'] = 2

In [152]:
gdf.shape

(35, 6)

In [413]:
wd_df = get_all_wd(num=4)

In [76]:
set_dict = dict(zip(wd_df.wetdet_delin_number, wd_df.SetID))

In [375]:
gdf.loc[:,'SetID'] = gdf.wdID.map(set_dict)

In [385]:
sa_gdf_all = get_all_SA(num=4)

In [386]:
all_gdf = pd.concat([sa_gdf_all[['wdID', 'code', 'SetID', 'geometry']],
                    gdf[['wdID', 'code', 'SetID', 'geometry']]], ignore_index=True)

In [88]:
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 9.0 minutes to complete


In [89]:
start = time.time()
all_mapIdx = read_all_mapIdx()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 1.0 minutes to complete


In [387]:
wdID_list, n = check_duplicates(sa_gdf_all.wdID.values)
wdID_list

['WD2019-0511', 'WD2021-0686', 'WD2022-0026']

In [388]:
wdID_list, n = check_duplicates(gdf.wdID.values)
wdID_list

[]

In [389]:
wdID_list, n = check_duplicates(all_gdf.wdID.values)

In [390]:
wdID_list

['WD2019-0259',
 'WD2019-0511',
 'WD2021-0686',
 'WD2018-0352',
 'WD2022-0026',
 'WD2019-0300',
 'WD2021-0495']

In [391]:
gdf.loc[gdf.wdID=='WD2021-0495', 'wdID'] = 'WD2021-0031'
gdf = gdf[~gdf.wdID.isin(['WD2019-0259', 'WD2018-0352', 'WD2019-0300'])]
sa_gdf_all = sa_gdf_all[~sa_gdf_all.wdID.isin(['WD2019-0259'])]
sa_gdf_all = sa_gdf_all[~((sa_gdf_all.wdID=='WD2021-0686') & (sa_gdf_all.SetID==1))]
sa_gdf_all = sa_gdf_all[~((sa_gdf_all.wdID=='WD2022-0026') & (sa_gdf_all.SetID==2))]

In [392]:
sa_gdf_all[sa_gdf_all.wdID.isin(wdID_list)]

,code,wdID,geometry,SetID
453,0,WD2019-0511,"POLYGON ((667916.072 1236023.094, 667881.278 1...",1
1120,2,WD2018-0352,MULTIPOLYGON Z (((590189.143 1033810.888 0.000...,2
1203,0,WD2019-0511,"MULTIPOLYGON Z (((598409.865 958092.642 0.000,...",2
2148,0,WD2019-0300,"POLYGON ((754471.158 1395046.121, 754571.146 1...",4
2183,0,WD2019-0511,"MULTIPOLYGON (((601741.516 983380.405, 601794....",4
2412,2,WD2021-0495,"POLYGON ((1184166.533 926606.419, 1184863.180 ...",4
2447,2,WD2021-0686,"POLYGON ((1164530.417 959846.060, 1164454.040 ...",4
2454,2,WD2022-0026,"POLYGON ((880278.127 1346054.362, 878422.925 1...",4


In [103]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [393]:
wd_df.loc[wd_df.county == 'TIllamook', 'county'] = 'Tillamook'
wd_df.loc[wd_df.county == 'Yamhill and Washington', 'county'] = 'Yamhill'

all_gdf = all_gdf.dissolve('wdID')
all_gdf['wdID'] = all_gdf.index
all_gdf.reset_index(drop=True, inplace=True)

In [394]:
selectedID = wd_df.parcel_id.astype(str) != 'nan'
wd_df = wd_df.copy()[selectedID]
wd_df['notes'] = wd_df.copy()['parcel_id'].apply(lambda x: make_notes(x))
wd_dt = wd_df[wd_df.notes == '']

In [424]:
wd_df_c = get_corrected_wd_df(num=4)

In [429]:
wd_df_c.loc[wd_df_c.county == 'Yamhill and Washington', 'county'] = 'Yamhill'

In [433]:
wd_df_c.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [432]:
start = time.time()
out = split_SA_by_rid_in_df(wd_df=wd_df_c[wd_df_c.notes.isnull()], 
                            sa_gdf_all=all_gdf, 
                            all_mapIdx=all_mapIdx, 
                            all_taxlot=all_taxlot, 
                            em_wids=all_gdf.wdID.unique(),
                            export=True, 
                            outnm='wd_sa_gdf', 
                            review=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 3.0 minutes to complete


In [434]:
# wd IDs to review
wdIDs = wd_df_c[~wd_df_c.notes.isnull()].wetdet_delin_number.unique()

In [454]:
len(wdIDs)

769

In [439]:
out.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'doc_name', 'doc_link',
       'isbatfile', 'status_nm', 'receiveddt', 'responsedt', 'reissuance',
       'project_id', 'site_id', 'SetID', 'record_ID', 'ORMapNum', 'geometry',
       'code', 'lat', 'lon'],
      dtype='object')

In [440]:
out_gdf = out[~out.wdID.isin(wdIDs)]

In [442]:
out.shape

(2232, 26)

In [441]:
out_gdf.shape

(2106, 26)

In [448]:
out_gdf.to_file(f'{outpath}\\test\\wd_sa_gdf.shp')

In [438]:
all_gdf_s = all_gdf[all_gdf.wdID.isin(wdIDs)]

In [445]:
len(all_gdf_s.wdID.unique())

763

In [450]:
'WD2017-0229' in all_gdf.wdID.unique()

False

In [451]:
'WD2021-0179' in all_gdf.wdID.unique()

False

In [449]:
all_gdf_s.head(3)

,wdID,code,SetID,geometry
0,WD2017-0003,2.0,1,"POLYGON ((447632.599 1578055.844, 447349.923 1..."
1,WD2017-0014,2.0,1,"POLYGON ((659383.287 224983.480, 659986.729 22..."
5,WD2017-0034,2.0,1,"POLYGON ((673696.996 1156836.787, 673762.888 1..."


In [453]:
wd_df_cor = get_corrected_wd_df(num=4)

In [457]:
wdIDs_s = [wid for wid in wdIDs if check_duplicates(wd_df_cor.loc[wd_df_cor.wetdet_delin_number==wid, 'trsqq'].values)[1] > 0]

In [460]:
wdIDs_ss = [wid for wid in wdIDs_s if len(wd_df_cor.loc[wd_df_cor.wetdet_delin_number==wid, 'county'].unique()) > 1]

In [458]:
len(wdIDs_s)

84

In [461]:
len(wdIDs_ss)

1

In [467]:
wd_df_cor.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [469]:
wd_df_cor[(wd_df_cor.wetdet_delin_number==wdIDs_s[0]) & (wd_df_cor.trsqq=='07N10W15BB')]

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot,SetID
91,WD2020-0280,07N10W15BB,"100,200,1600,1700,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,132949,92,"ROW, Partial",2020,2020,N,1
95,WD2020-0280,07N10W15BB,"3200,3300,ROWs all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139666,96,"ROW, Partial",2020,2020,N,1
97,WD2020-0280,07N10W15BB,"3400,3500,3600,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139665,98,"ROW, Partial",2020,2020,N,1
99,WD2020-0280,07N10W15BB,"2700,2800,2900,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139663,100,"ROW, Partial",2020,2020,N,1
100,WD2020-0280,07N10W15BB,"2400,2500,2600,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139662,101,"ROW, Partial",2020,2020,N,1
101,WD2020-0280,07N10W15BB,"1800,3700 all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139660,102,Partial,2020,2020,N,1
103,WD2020-0280,07N10W15BB,"3000,3100,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139664,104,"ROW, Partial",2020,2020,N,1
104,WD2020-0280,07N10W15BB,"2100,2200,2300,ROW all partial",W of 90266 Hwy 101,Warrenton,Clatsop,Dune Estates Subdivision,WD2020-0280 delineation,46.094286,-123.92086,...,2020-10-22,NaN,79733.0,139661,105,"ROW, Partial",2020,2020,N,1


In [466]:
get_point_from_lonlat(lon=-123.92086, lat=46.094286, transprj=True, export=True)

,Longitude,Latitude,geometry
0,-123.92086,46.094286,POINT (444428.530 1601763.211)


In [462]:
wd_df_cor[wd_df_cor.wetdet_delin_number==wdIDs_ss[0]]

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot,SetID
2012,WD2022-0026,02S04E02A,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,138237,2013,Many,2022,2022,Y,2
2013,WD2022-0026,01S05E00,Many,Bull Run Reservoir #2,NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 offsite wetdet,45.444581,-122.154343,...,2022-01-27,NaN,83419.0,138236,2014,Many,2022,2022,Y,2
2014,WD2022-0026,01S06E00,Many,Bull Run Reservoir #1,NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 wetdet,45.471494,-122.095475,...,2022-01-27,NaN,83419.0,138222,2015,Many,2022,2022,Y,2
2015,WD2022-0026,02S05E04,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141443,2016,Many,2022,2022,Y,2
2016,WD2022-0026,02S05E03,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141444,2017,Many,2022,2022,Y,2
2017,WD2022-0026,01S05E33,Many,Bull Run Reservoir #2,NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 offsite wetdet,45.444581,-122.154343,...,2022-01-27,NaN,83419.0,141445,2018,Many,2022,2022,Y,2
2018,WD2022-0026,02S04E01AC,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141447,2019,Many,2022,2022,Y,2
2019,WD2022-0026,02S05E06,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141441,2020,Many,2022,2022,Y,2
2020,WD2022-0026,02S05E05,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141442,2021,Many,2022,2022,Y,2
2021,WD2022-0026,02S04E01,Many,"12852 SE Bluff Rd, Sandy, OR",NaN,Clackamas,Clackamas Mt Hood EHP BRIC,WD2022-0026 - PGE,45.430128,-122.275735,...,2022-01-27,NaN,83419.0,141446,2022,Many,2022,2022,Y,2


In [415]:
cnt_df = wd_df.county.value_counts().rename_axis('county').reset_index(name='counts')

In [403]:
dat = pd.read_excel(wdpath+'\\202307RunPolkGilliamUmatillaUnionWheeler2017-2022ALL.xlsx')

In [405]:
cnt_df1 = dat.county.value_counts().rename_axis('county').reset_index(name='counts')

In [416]:
cnt_df2 = pd.concat([cnt_df, cnt_df1], ignore_index=True)

In [418]:
cnt_df2.to_csv(inpath+'\\reporting\\deliveries_records_by_county.csv', index=False)